In [83]:
#Import các thư viện
import requests
from datetime import datetime as dt
import pandas as pd
from dateutil import relativedelta
import sys
import os
import pygsheets
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pygsheets
import time
from bs4 import BeautifulSoup
import glob
import json

In [84]:
#Mở gg sheet để lấy ra dữ liệu cũ
client = pygsheets.authorize(service_file = "/Users/genkin/Downloads/MAIDO BOT/maido-agency-e7cc6f08cb95.json")
sh = client.open('(MD-GK) RECRUITING MANAGEMENT')
wks = sh.worksheet_by_title('New CV')
recruitment_data = wks.get_as_df()
print('Get old data')

Get old data


In [85]:
#Cài đặt option cho Chrome driver -> mở Chrome -> Mở web 
options = Options()
options.add_experimental_option('prefs', {
"download.default_directory": r"/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded", #Change default directory for downloads
"download.prompt_for_download": False, #To auto download the file
"download.directory_upgrade": True,
"plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
driver = webdriver.Chrome(options = options)
driver.get('https://careerbuilder.vn/vi/employers/hrcentral/manageresume')

In [86]:
#Tìm ô input tài khoản, mật khẩu và truyền tài khoản mật khẩu từ file login.json vào. Sau đó clink vào nút đăng nhập.
f = open('login.json')

data = json.load(f)

username = data['username']
password = data['password']

user_field = driver.find_element(By.XPATH, '//*[@id="frmLogin"]/div[1]/div[2]/input')
user_field.send_keys(username)
time.sleep(5)

pass_field = driver.find_element(By.XPATH, '//*[@id="frmLogin"]/div[2]/div[2]/input')
pass_field.send_keys(password)
time.sleep(5)

login_field = driver.find_element(By.XPATH, '//*[@id="frmLogin"]/div[3]/div/button')
login_field.click()
time.sleep(5)
print('Login')

Login


In [87]:
#Lặp qua các job để lấy về đường link -> id của job
job_link = []

for i in range(2,100):
    # Bắt đầu từ job thứ 2
    try:
        job_input = driver.find_element(By.XPATH,'//*[@id="select-folder"]')
        job_input.click()
        job =  driver.find_element(By.XPATH, f'//*[@id="select-folder"]/option[{i}]')
        job.click()
        url = driver.current_url
        index = str(url).find('/*/0/0/*/*/7/2/6/2/0/desc/hr.1661228713/1')
        id = str(url)[index-8:index]
        link = r'https://careerbuilder.vn/vi/employers/hrcentral/manageresume/1/' + id +r'/*/0/0/03-02-2023/*/7/2/6/2/0/desc/hr.1661228713/1'
        job_link.append(link)
    
    #Khi đã hết job thì quay về job đầu tiên
    except:
        job_input = driver.find_element(By.XPATH,'//*[@id="select-folder"]')
        job_input.click()
        job =  driver.find_element(By.XPATH, f'//*[@id="select-folder"]/option[1]')
        job.click()
        url = driver.current_url
        index = str(url).find('/*/0/0/*/*/7/2/6/2/0/desc/hr.1661228713/1')
        id = str(url)[index-8:index]
        link = r'https://careerbuilder.vn/vi/employers/hrcentral/manageresume/1/' + id +r'/*/0/0/03-02-2023/*/7/2/6/2/0/desc/hr.1661228713/1'
        job_link.append(link)
        break



In [88]:
#Lặp qua từng đường link của job lấy về thông tin ứng viên
candidate_url = []

for link in job_link:

    driver.get(link)
    
    #Lấy ra có bao nhiêu ứng viên
    page_navi = driver.find_element(By.XPATH, f'//*[@id="tab-1"]/div/div[2]/div[2]/div/p').text
    if page_navi != "":
        no_candidate = int(page_navi.split(' ')[6])

        #Tính toán số page
        if no_candidate % 20 == 0:
            no_page = no_candidate//20
        else:
            no_page = (no_candidate//20) + 1

        #Khởi tạo các page
        page_lst = [i for i in range(1,no_page +1)]
        
        #Nếu có nhiều hơn 1 page thì lần lượt click vào từng page
        if len(page_lst) >1:
            for page in page_lst:
                page = driver.find_element(By.XPATH,f'//*[@id="tab-1"]/div/div[3]/div[2]/div/ul/li[{page}]/a')
                page.click()
                pagesource = BeautifulSoup(driver.page_source)
                candidate = pagesource.find_all('a',class_ = 'name')
                for i in candidate:
                    up_index = str(i).find('href="') + 6
                    low_index = str(i).find('" target=')
                    candidate_url.append(str(i)[up_index:low_index])
        
        #Nếu chỉ có 1 page thì lấy thông tin luôn
        else:
            pagesource = BeautifulSoup(driver.page_source)
            candidate = pagesource.find_all('a',class_ = 'name')
            for i in candidate:
                up_index = str(i).find('href="') + 6
                low_index = str(i).find('" target=')
                candidate_url.append(str(i)[up_index:low_index])
candidate_url = list(set(candidate_url))
print('Get candidate url:', len(candidate_url))

Get candidate url: 37


In [89]:
#Loại ra những ứng viên đã lấy về (so sánh id)
old_values = recruitment_data['id'].values
new_candidate = []
for i in candidate_url:
    if i[-17:-9] not in old_values:
        new_candidate.append(i)
print('New candidate start from 2023-02-06:', len(new_candidate))

New candidate start from 2023-02-06: 4


In [90]:
#Loại bỏ dấu trong tên
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

In [91]:
#Tạo list rỗng để chứa các giá trị
position = []
name = []
mail = []
phone_number = []
company = []
id = []
year = []
month = []
date = []

#Lặp qua các đừng dẫn đến ứng viên mới để lấy ra thông tin
for candidate in new_candidate:
    driver.get(candidate)

    candidate_position = driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[1]/nav/ol/li[2]').text
    position.append(candidate_position)
    if candidate_position.lower() == 'accountant' or candidate_position.lower() == 'customer service associate':
        company.append('Genkin')
    else:
        company.append('Maido')

    candidate_name = driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[1]/div[1]/ul/li[1]/p[2]').text
    name.append(candidate_name)

    apply_date = driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[3]/ul/li[3]/p[2]').text
    d = apply_date.split('-')[0]
    m = apply_date.split('-')[1]
    y = apply_date.split('-')[2]
    date.append(f'{y}-{m}-{d}')
    month.append(m)
    year.append(y)

    #Nếu thông tin về sdt bị thiếu thì vị trí mail sẽ thay đổi. 
    if driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[2]/div/ul/li[1]/p[1]/strong').text.strip() == "ĐTDĐ :":
        phone_number.append(driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[2]/div/ul/li[1]/p[2]').text)
        mail.append(driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[2]/div/ul/li[2]/p[2]').text)
    else:
        phone_number.append("")
        mail.append(driver.find_element(By.XPATH, '//*[@id="tab-1"]/div/div/div/div[4]/div[1]/div/div[2]/div[1]/div[2]/div/ul/li/p[2]').text)

    #Truy xuất id từ link
    candidate_id = candidate[-17:-9]
    id.append(candidate_id)
print('Get data')

Get data


In [92]:
df =pd.DataFrame({'date':date,
                  'month':month,
                  'year':year,
                  'company': company,
                  'name':name,
                  'mail':mail,
                  'phone_number':phone_number,
                  'position':position,
                  'id':id
                 }, columns = ['id','date','month','year','company','name','mail','phone_number','position'])

In [ ]:
df


In [94]:
#Tải file cv về máy
output_dir = '/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded'

for i in range(0,len(df)):

    candidate_name = df.iloc[i]['name']
    candidate_id = df.iloc[i]['id']
    candidate_position = df.iloc[i]['position']
    apply_date = df.iloc[i]['date']
    
    download_url = f'https://careerbuilder.vn/vi/employers/popup/downloadresume?resume_id={candidate_id}'
    driver.get(download_url)
time.sleep(30)
print('CV downloaded')

CV downloaded


In [95]:
#Tạo ra 1 dictionary để đổi tên
new_name = ['_'.join(position.upper().split(' ')) +"("+date+')' for position,date in zip(df['position'],df['date'])]
id = df['id'].values

rename = {}

for i in range(0,len(df)):
    rename[id[i]] = new_name[i]


In [96]:
#Quét các file hiện có trong folder và đổi tên nếu match với dict đã tạo.
upload_file_list = []
file_name_dict = {}
list_of_files = glob.glob('/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/*.pdf')
for i in list_of_files:
    index = str(i)[-12:-4]
    if index in rename.keys():
        new_name = str(i).replace(index,rename[index]).replace('.pdf','')

        #Kiểm tra tên mới đã tồn tại hay chưa
        if(not os.path.exists(new_name)):    
            os.rename(i,new_name)
        file_name = os.path.basename(new_name)
        file_name_dict[index] = file_name
        upload_file_list.append(new_name)
print(f'{len(upload_file_list)} downloaded files have renamed')

4 downloaded files have renamed


In [97]:
list_of_files

['/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/Tinh_Tinh_36BB574F.pdf',
 '/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/Nguyen_Phuoc_Thien_36BBA209.pdf',
 '/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/Chau_Le_Phuc_36BB8CC9.pdf',
 '/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/Ly_Huynh_To_Nhu_36BB42A2.pdf',
 '/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/Shunn_36BB8E6C.pdf',
 '/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/Chu_Thanh_Thien_36BCB285.pdf',
 '/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/nguyen_hoang_36BB3C34.pdf',
 '/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/Le_Thi_Hanh_Yen_36BCAF87.pdf',
 '/Users/genkin/Downloads/CV_CRAWL_BOT/Job_Application_Crawl-main/CV_downloaded/Nguyen_Chinh_36BBBF8A.pdf',
 '/Users/genkin/Downloa

In [98]:
#Upload lên gg drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()    
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)
print('Authorization done')
targetDirID = '122XoM5A0SZWBqhFshmMTuem3Xfm-npuo'
 
exist_file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(targetDirID)}).GetList()

#Nếu file bị trùng thì upload rồi xóa file cũ
for upload_file in upload_file_list:
	if(not os.path.exists(upload_file)):
		continue
		
	fileName = os.path.basename(upload_file)
	for file1 in exist_file_list:
		if file1['title'] == fileName:
			file1.Delete()
		
	gfile = drive.CreateFile({'parents': [{'id': targetDirID}], 'title': fileName})
	# Read file and set it as the content of this instance.
	gfile.SetContentFile(upload_file)
	gfile.Upload() # Upload the file.
print(f'{len(upload_file_list)} files uploaded')

Authorization done
4 files uploaded


In [99]:
#Lấy ra các tên + đường dẫn các file hiện có trong folder trên drive
name_lst = []
cv_link = []
files = drive.ListFile({"q": "'" + targetDirID + "' in parents and mimeType!='application/vnd.google-apps.folder'"}).GetList()
for file in files:
    keys = file.keys()
    if file['shared'] and 'alternateLink' in keys:
        link = file['alternateLink']
    else:
        link = 'No Link Available. Check your sharing settings.'
    name = file['title']
    name_lst.append(name)
    cv_link.append(link)
    
cv_df = pd.DataFrame({'file_name':name_lst,'cv':cv_link}, columns=['file_name','cv'])
cv_df.drop_duplicates(subset='file_name', keep="last",inplace = True)

In [ ]:
cv_df

In [ ]:
df

In [102]:
#Nối link cv với dataframe chứa thông tin
df['file_name'] = [file_name_dict[i] for i in df['id']]
data = pd.merge(df,cv_df,how='left')
data.drop_duplicates(inplace = True)
print(f'{len(data)} new data to append')

4 new data to append


In [103]:
#Làm sạch/ thêm 1 vài cột cần thiết
data['source'] = ['CareerBuilder']*len(data)
data['graduated'] = ['']*len(data)
data['note'] = ['']*len(data)
data['name'] = [str(i).upper() for i in data['name']]
data['company'] = [str(i).upper() for i in data['company']]
data['position'] = [str(i).upper() for i in data['position']]

In [104]:
#Ghi lên gg sheet
append_index = "A" + str(len(recruitment_data)+2)

if len(data) > 0:
    wks.set_dataframe(data[['id','company','year','month','date','name','source','position','phone_number','mail','cv','graduated','note']], start=append_index, copy_head=False)
print('Done')

Done
